In [ ]:
import requests
import zipfile
from collections import Counter
import os

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
import xgboost as xgb
import shap
import matplotlib.pyplot as plt
import numpy as np

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 20)


In [ ]:
if not os.path.exists('fetal-health-classification.zip'):
    r = requests.get('https://www.kaggle.com/api/v1/datasets/download/andrewmvd/fetal-health-classification')
    with open('fetal-health-classification.zip', 'wb') as f:
        f.write(r.content)
    
if not os.path.exists('data/fetal_health.csv'):
    with zipfile.ZipFile('fetal-health-classification.zip', 'r') as zip_ref:
        zip_ref.extractall('data')


df = pd.read_csv(r"data/fetal_health.csv")
df.head()

In [ ]:
X = df.drop('fetal_health', axis=1)
# Map classes 1,2,3 -> 0,1,2
y = df['fetal_health'] - 1  # subtract 1 from all labels

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)


In [ ]:
X_test[:1] , y_test[:1]

In [ ]:
X[:1]

In [ ]:
# Check original distribution
print("Before SMOTE:", Counter(y_train))

# Apply SMOTE
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# Check new distribution
print("After SMOTE:", Counter(y_train_res))


In [ ]:
model = xgb.XGBClassifier(
    objective='multi:softmax',
    # objective='multi:softprob',  # outputs probabilities
    num_class=3,
    eval_metric='mlogloss',
    use_label_encoder=False,
    random_state=42
)

# Train on SMOTE-resampled data
model.fit(X_train_res, y_train_res)

# Evaluate
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
# save the trained model
model.save_model("models/fetal_xgb_model.json")

In [ ]:
explainer = shap.Explainer(model.predict_proba, X_test)
shap_values = explainer(X_test)

In [ ]:
shap.save_html("output/shap_fetal_health.html", shap_values)

In [ ]:
shap.summary_plot(shap_values, X_test, max_display=10)

In [ ]:
# The shap_values object from the previous cell already contains the values for each class
# We can directly use it for plotting

# Plot the summary plot with feature importance for each class
shap.summary_plot(shap_values, X_test, plot_type="bar")

In [ ]:
N = 50  # number of samples to plot
samples = np.arange(N)
# Probabilities for each class
y_proba = model.predict_proba(X_test)

plt.figure(figsize=(12,6))
plt.bar(samples, y_proba[:N,0], label='Class 0 (Normal)')
plt.bar(samples, y_proba[:N,1], bottom=y_proba[:N,0], label='Class 1 (Suspect)')
plt.bar(samples, y_proba[:N,2], bottom=y_proba[:N,0]+y_proba[:N,1], label='Class 2 (Pathological)')

plt.xlabel('Test Samples')
plt.ylabel('Predicted Probability')
plt.title('Predicted Probabilities for Test Samples')
plt.legend()
plt.show()


In [ ]:
# Evaluate
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
# Map numerical classes to labels
class_map = {0: "Normal", 1: "Suspect", 2: "Pathological"}

def patient_risk_with_recommendation(patient_data: pd.DataFrame):
    """
    patient_data: single-row dataframe with same features as training set
    Returns predicted class, top 3 SHAP features, and patient recommendation
    """
    # 1️⃣ Predict probabilities and class
    probs = model.predict_proba(patient_data)[0]
    pred_class_idx = np.argmax(probs)
    pred_class_name = class_map[pred_class_idx]

    # 2️⃣ Compute SHAP values for this patient
    shap_values = explainer(patient_data)
    shap_vals_for_class = shap_values[:, :, pred_class_idx].values[0]

    # 3️⃣ Create a dataframe for features and SHAP values
    feature_df = pd.DataFrame({
        "feature": patient_data.columns,
        "shap_value": shap_vals_for_class,
        "feature_value": patient_data.iloc[0].values
    }).sort_values(by="shap_value", key=abs, ascending=False)

    # 4️⃣ Top 3 most important features
    top_features = feature_df.head(3)

    # 5️⃣ Generate recommendations based on SHAP values and class
    recommendations = []
    for idx, row in top_features.iterrows():
        feature, value, shap_val = row['feature'], row['feature_value'], row['shap_value']
        if pred_class_name == "Normal":
            # Positive features reassuring
            if shap_val > 0:
                recommendations.append(f"{feature} ({value}) supports normal fetal health.")
            else:
                recommendations.append(f"{feature} ({value}) slightly reduces reassurance, monitor routinely.")
        elif pred_class_name == "Suspect":
            if shap_val > 0:
                recommendations.append(f"{feature} ({value}) increases risk; closer monitoring recommended.")
            else:
                recommendations.append(f"{feature} ({value}) slightly reduces risk, but patient still at suspect level.")
        elif pred_class_name == "Pathological":
            if shap_val > 0:
                recommendations.append(f"{feature} ({value}) strongly indicates high risk; urgent monitoring/intervention required.")
            else:
                recommendations.append(f"{feature} ({value}) reduces risk but patient still at pathological level.")

    # 6️⃣ Output
    return {
        "predicted_class": pred_class_name,
        "predicted_probabilities": probs,
        "top_features": top_features,
        "recommendations": recommendations
    }

# --------------------------
# Example usage
# --------------------------
patient_example = X_test.iloc[[0]]  # pick one patient from test set
result = patient_risk_with_recommendation(patient_example)

print("Predicted Class:", result['predicted_class'])
print("Predicted Probabilities:", result['predicted_probabilities'])
print("\nTop Features Driving Prediction:\n", result['top_features'])
print("\nRecommendations:\n", "\n".join(result['recommendations']))


In [ ]:
def shap_based_recommendations(shap_values, X, class_names):
    # shap_values.values shape: (n_samples, n_features, n_classes)
    # Compute mean absolute SHAP per feature per class
    mean_shap = np.abs(shap_values.values).mean(axis=0)  # shape: (n_features, n_classes)
    mean_shap_df = pd.DataFrame(mean_shap, index=X.columns, columns=class_names)

    rec_dict = {}
    for feature in X.columns:
        # Identify class with max influence
        max_class = mean_shap_df.loc[feature].idxmax()

        # Map to recommendation
        if feature == "accelerations":
            rec = "Encourage maternal movement and fetal monitoring to improve fetal heart accelerations"
        elif feature == "abnormal_short_term_variability":
            rec = "Schedule detailed fetal monitoring; consult obstetrician for possible interventions"
        elif feature == "histogram_mean":
            rec = "Ensure regular prenatal check-ups to monitor overall fetal health metrics"
        elif feature == "histogram_number_of_zeroes":
            rec = "No immediate action; feature minimally impacts risk"
        elif feature == "uterine_contractions":
            rec = "Track contractions and report abnormalities; maintain hydration and rest"
        elif feature == "baseline_value":
            rec = "Monitor baseline fetal heart rate; consult doctor if deviations persist"
        else:
            rec = "Regular monitoring recommended"

        rec_dict[feature] = {"most_influential_class": max_class, "recommendation": rec}

    return rec_dict

# Apply
recommendations = shap_based_recommendations(shap_values, X_test, ["Normal","Suspect","Pathologic"])

# Example output
for feat, info in recommendations.items():
    print(f"Feature: {feat} | Class impact: {info['most_influential_class']} | Recommendation: {info['recommendation']}")


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

tokenizer = AutoTokenizer.from_pretrained("AdaptLLM/medicine-LLM")
model = AutoModelForCausalLM.from_pretrained("AdaptLLM/medicine-LLM").to("cuda")

llm = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
import shap
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline


new_data = pd.DataFrame([{
    "baseline value": 1953,
    "accelerations": 133.0,
    "fetal_movement": 0.0,
    "uterine_contractions": 0.0,
    "light_decelerations": 0.012,
    "severe_decelerations": 0.001,
    "prolongued_decelerations": 0.0,
    "abnormal_short_term_variability": 0.002,
    "mean_value_of_short_term_variability": 60.0,
    "percentage_of_time_with_abnormal_long_term_variability": 3.0,
    "mean_value_of_long_term_variability": 0.0,
    "histogram_width": 0.0,
    "histogram_min": 97.0,
    "histogram_max": 58.0,
    "histogram_number_of_peaks": 155.0,
    "histogram_number_of_zeroes": 4.0,
    "histogram_mode": 0.0,
    "histogram_mean": 125.0,
    "histogram_median": 96.0,
    "histogram_variance": 105.0,
    "histogram_tendency": 79.0
}])

# -----------------------
# 2️⃣ Load trained XGBoost model
# -----------------------
# model = xgb.XGBClassifier()
# model.load_model("fetal_xgb_model.json")

# -----------------------
# 3️⃣ Predict class
# -----------------------


In [ ]:
# Requires: transformers, sentence-transformers, faiss-cpu, torch, shap, xgboost, numpy, pandas
# pip install transformers sentence-transformers faiss-cpu

from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import json
import textwrap
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# ---------- Utility: top-N features by mean absolute SHAP ----------
def top_n_shap_features(shap_values, feature_names, n=8):
    # shap_values.values shape: (n_samples, n_features, n_classes)
    mean_abs = np.abs(shap_values.values).mean(axis=0)  # (n_features, n_classes)
    mean_per_feature = mean_abs.sum(axis=1)            # (n_features,)
    idx = np.argsort(-mean_per_feature)[:n]
    return [feature_names[i] for i in idx], mean_per_feature[idx]

# ---------- Build vector DB for documents (run once) ----------
def build_faiss_index(docs, embed_model_name="sentence-transformers/all-MiniLM-L6-v2"):
    embedder = SentenceTransformer(embed_model_name)
    texts = []
    meta = []
    for i, d in enumerate(docs):
        # split long doc into paragraphs
        for j, para in enumerate(d.split("\n\n")):
            para = para.strip()
            if len(para) < 50:
                continue
            texts.append(para)
            meta.append({"doc_id": i, "para_index": j, "text": para})
    embeddings = embedder.encode(texts, convert_to_numpy=True, show_progress_bar=False)
    dim = embeddings.shape[1]
    index = faiss.IndexFlatIP(dim)
    faiss.normalize_L2(embeddings)
    index.add(embeddings)
    return index, embeddings, meta, embedder

# ---------- Retrieve top-k relevant snippets for a query ----------
def retrieve_snippets(query, index, embeddings, meta, embedder, top_k=3):
    q_emb = embedder.encode([query], convert_to_numpy=True)
    faiss.normalize_L2(q_emb)
    D, I = index.search(q_emb, top_k)
    snippets = []
    for idx in I[0]:
        if idx < len(meta):
            snippets.append(meta[idx]["text"])
    return snippets

# ---------- Build concise prompt with strict grounding rules ----------
def build_prompt(pred_class, pred_prob, top_features, shap_summary, feature_values, snippets_map, docs_list):
    system = (
        "You are a medical assistant restricted to use ONLY the provided documents. "
        "Do NOT add facts not present in those documents. If you cannot find support for a recommendation, respond 'no source found'. "
        "Answer concisely and output only valid JSON (array of objects) following schema: "
        "[{feature, recommendation, RAG, source}]."
    )
    # concise SHAP summary for top features
    shap_lines = []
    for f in top_features:
        vals = shap_summary[f]
        # compute mean magnitude and main direction (which class has largest abs)
        arr = np.array(vals)
        main_class = int(np.argmax(np.abs(arr)))
        mean_mag = float(np.mean(np.abs(arr)))
        shap_lines.append(f"{f}: main_class={main_class}, mean_abs={mean_mag:.4f}")
    shap_text = "\n".join(shap_lines)

    # attach only small context for each feature (snippets_map)
    snippets_text = ""
    for f in top_features:
        snippets = snippets_map.get(f, [])
        if snippets:
            snippets_text += f"\n--- {f} supporting snippets ---\n"
            for s in snippets:
                snippets_text += textwrap.shorten(s, width=400, placeholder="...") + "\n"

    prompt = (
        f"SYSTEM:\n{system}\n\n"
        f"MODEL PREDICTION: class={pred_class}, probs={np.round(pred_prob,4).tolist()}\n\n"
        f"TOP FEATURES (SHAP):\n{shap_text}\n\n"
        f"FEATURE VALUES:\n{json.dumps(feature_values, indent=None)}\n\n"
        f"{snippets_text}\n\n"
        "TASK:\nFor each of the listed TOP FEATURES, produce one short actionable recommendation for the pregnant woman, "
        "assign RAG (Red/Amber/Green) using SHAP influence (higher mean_abs -> higher priority), and give the exact snippet text used as source. "
        "Return ONLY JSON array. If none of the provided snippets support an action, set source = 'no source found'."
    )
    return prompt

# ---------- Example usage within your existing flow ----------
# 1) your shap_values, new_data, model prediction exist
feature_names = list(new_data.columns)
top_features, mags = top_n_shap_features(shap_values, feature_names, n=8)

# 2) load/prepare documents list (use the filled documents_text split into items)
documents_list = [
    "WHO guideline excerpt about fetal monitoring and hydration: Continuous fetal monitoring is advised when there is reduced variability...",
    "ACOG Practice Bulletin excerpt: Baseline fetal heart rate should remain between 110-160 bpm. Frequent decelerations suggest hypoxia...",
    "PubMed excerpt about short-term variability correlation with hypoxia: Low short-term variability indicates possible hypoxia or acidosis..."
    # add more long docs as needed
]

# 3) build faiss index (do once, keep index and embedder)
index, embeddings, meta, embedder = build_faiss_index(documents_list)

# 4) for each top feature, retrieve snippets (simple query = feature name + 'fetal monitoring')
snippets_map = {}
for f in top_features:
    q = f + " fetal monitoring implications"
    snippets_map[f] = retrieve_snippets(q, index, embeddings, meta, embedder, top_k=2)

# 5) build prompt
prompt = build_prompt(pred_class, pred_prob, top_features, shap_summary, feature_values, snippets_map, documents_list)

# # 6) load your local medical LLM (longer-context model preferred) and generate deterministically
# tokenizer = AutoTokenizer.from_pretrained("your-medical-llm-path", use_fast=True)
# model_llm = AutoModelForCausalLM.from_pretrained("your-medical-llm-path", device_map="auto", torch_dtype=torch.float16)
gen = pipeline("text-generation", model=model, tokenizer=tokenizer)

# ensure prompt length safe
tokens = len(tokenizer.encode(prompt))
max_allowed = tokenizer.model_max_length - 256
if tokens > max_allowed:
    raise ValueError(f"Prompt too long ({tokens} tokens). Reduce docs or top_features.")

out = gen(prompt, do_sample=False, max_new_tokens=512)[0]["generated_text"]

# parse JSON in output robustly
try:
    # attempt to extract JSON array from output
    start = out.find("[")
    end = out.rfind("]") + 1
    json_text = out[start:end]
    recommendations = json.loads(json_text)
except Exception:
    recommendations = {"error": "LLM output not valid JSON", "raw": out}

print(recommendations)


In [ ]:
response = llm(prompt, do_sample=False)[0]['generated_text']

# -----------------------
# 9️⃣ Display results
# -----------------------
print("---- LLM Recommendations ----")
print(response)